# noaa-weather-hourly
This script cleans and formats a manually downloaded "National Oceanic and Atmospheric Administration" (NOAA) "LCD CSV" weather file.  This is the development version of the script, not the publishable version.

 __Originated From:__
https://github.com/emskiphoto/Process-historical-NOAA-LCD-weather<BR>
    
Copyright Matt Chmielewski<BR>
https://github.com/emskiphoto/noaa-weather-hourly<BR>
January 1, 2025

### Load Python packages

In [1]:
# direct the jupyter notebook development script to add package path to PYTHONPATH
# to allow for loading of source modules.
import sys
sys.path.append('../noaa-weather-hourly')

In [2]:
# import modules specific to this package 
from config import *
from utils import *

In [3]:
import pandas as pd
import csv
import pathlib
import re
# turn off Jedi autocomplete (that was causing more problems than benefits post Win10 update 3-13-2020)
%config Completer.use_jedi = False 

### Parameters - Store in config.py

In [4]:
is_development = True

## Locations

In [5]:
dir_cwd = pathlib.Path.cwd()
dir_cwd

WindowsPath('C:/Users/user/OneDrive/python_envs/noaa-weather-hourly-cli/dev')

In [6]:
# using 'data' within 'dev' folder - potentially different from 'data' in package folder
# dir_data should be dir_cwd in production script
if is_development:
    dir_data = dir_cwd / 'data'
else:
    dir_data = dir_cwd
assert dir_data.is_dir()

#### Are there any .CSV files of any naming format?
If not, stop the script, there is nothing to do without the local .csv's.

In [7]:
# pretty name of directory
dir_data_posix = dir_data.as_posix()
# list of all .csv files in dir_data
dir_data_csv_files = sorted([f_.name for f_ in dir_data.glob('*.csv') if f_.is_file()])

In [8]:
# if some .csv files are present, continue.  Otherwise halt process and inform user.
try:
    assert len(dir_data_csv_files) >= 1
except:
    message_ = message_no_csv_files_found.format(dir_data_posix = dir_data_posix)
    print(message_)
#     assert
    raise
# string version of list of all csv files
dir_data_csv_files_str = ', '.join(dir_data_csv_files)

### Locate LCD .CSV file(s) 'files_lcd_input'
This script is intended to be executed from a terminal command line.  The LCD input file(s) are expected to be saved in the same directory that the command line is executed in.  The file name(s) are expected to match the pattern associated with multiple LCD file versions in 'patterns_lcd_input_files' (two versions currently).  However, if a file(s) with this pattern is not identifed, do NOT attempt to use any non-matching .CSV file in the same directory.  Inform user that no matching file was found and no files will be opened or created.

The benefits of this approach are:
1. code will not mistakenly use non-LCD files
2. User can be sloppy (or organized) with their LCD file storage.  New source files and output files can simply be accumulated in the same folder with no data loss.
3. Simple command line requires no mandatory input, only optional frequency and parameter setting inputs.

### which version of LCD files are avaialble and which are the most recent?
1. find all files that match v1 or v2 naming
2. find the most recent file
3. Determine if most recent file is v1 or v2 format 'lcd_version'
4. see if there is more than one file with the same station ID
5. create list 'files_lcd' with one or more lcd files of same station id 

In [9]:
# 1. find all files that match v1 or v2 naming and sort by last modified date descending
# create 'version_files' dictionary with LCD version number as key and list of matching 
# files as values
version_files = {v_ : find_files_re_pattern_sorted_last_modified(dir_data, pattern_) for
                 v_, pattern_ in version_pattern_lcd_input.items()}
version_files

{1: [WindowsPath('C:/Users/user/OneDrive/python_envs/noaa-weather-hourly-cli/dev/data/3876540.csv'),
  WindowsPath('C:/Users/user/OneDrive/python_envs/noaa-weather-hourly-cli/dev/data/3875753.csv')],
 2: [WindowsPath('C:/Users/user/OneDrive/python_envs/noaa-weather-hourly-cli/dev/data/LCD_USW00014939_2023.csv'),
  WindowsPath('C:/Users/user/OneDrive/python_envs/noaa-weather-hourly-cli/dev/data/LCD_USW00014939_2022.csv'),
  WindowsPath('C:/Users/user/OneDrive/python_envs/noaa-weather-hourly-cli/dev/data/LCD_USW00014939_2021.csv'),
  WindowsPath('C:/Users/user/OneDrive/python_envs/noaa-weather-hourly-cli/dev/data/LCD_USW00014939_2020.csv'),
  WindowsPath('C:/Users/user/OneDrive/python_envs/noaa-weather-hourly-cli/dev/data/LCD_USW00094846_2023.csv'),
  WindowsPath('C:/Users/user/OneDrive/python_envs/noaa-weather-hourly-cli/dev/data/LCD_USW00094846_2022.csv'),
  WindowsPath('C:/Users/user/OneDrive/python_envs/noaa-weather-hourly-cli/dev/data/LCD_USW00094846_2021.csv'),
  WindowsPath('C:/Us

In [10]:
files_pattern_match = [x for xs in version_files.values() for x in xs]

In [11]:
# what if no files were found?  
# return message and halt process
# which files matched lcd patterns, regardless of version or date?
# files_pattern_match = [x for xs in version_files.values() for x in xs]
try:
    assert len(files_pattern_match) >= 1
except:
    message_ = message_no_lcd_files_found.format(dir_data_posix = dir_data_posix,
                                 patterns_lcd_examples_str = patterns_lcd_examples_str,
                                dir_data_csv_files_str = dir_data_csv_files_str)
    print(message_)
#     assert
    raise

In [12]:
# find most recently modified file by lcd version
version_file_last_modified = {version_ : files_[0] for version_, files_ in version_files.items()}
version_file_last_modified

{1: WindowsPath('C:/Users/user/OneDrive/python_envs/noaa-weather-hourly-cli/dev/data/3876540.csv'),
 2: WindowsPath('C:/Users/user/OneDrive/python_envs/noaa-weather-hourly-cli/dev/data/LCD_USW00014939_2023.csv')}

In [13]:
# 2. find the most recent file
file_last_modified = sorted([(f, f.stat().st_mtime) for
                  f in version_file_last_modified.values()],
           key=lambda x: x[1], reverse=True)[0][0]
# 3. Determine if most recent file is v1 or v2 format 'lcd_version'
# versions start with '1' so need to add 1 to zero-indexed list
lcd_version = list(version_file_last_modified.values())\
                            .index(file_last_modified) + 1
# file_last_modified, lcd_version

In [14]:
# make sure we have the right version
assert file_last_modified in version_files[lcd_version]

### Group LCD .CSV input file(s) as 'files_lcd_input'

In [15]:
# 4. see if there is more than one file that has the same station ID
# as found in 'file_last_modified'
# This requires extraction of a unique identifier in LCD file name that is common to
# other LCD files for same location (but probably different dates).  

# Note that only the LCD v2 files need to be grouped.  LCD v1 files are 
# delivered with multi-year date ranges (if requested) while LCD v2
# files are for discrete calendar years (or less), for example 'LCD_USW00014939_2020.csv'.  

# Grouping LCD v1 files could be implemented, but this would require cooperation
# from the user in terms of renaming the LCD v1 files in a specific format.
# LCD v1 files are delivered with the same name ('3876540.csv') regardless
# of date range of data in file.    

In [16]:
# files_lcd_input - empty list to hold final, qualified selection of LCD input files
files_lcd_input = []
# different treatment for v2 LCD files
if lcd_version == 2:
# extract id_file_lcd2 as the blob of characters between first and second '_'
# reference 'LCD_USW00014939_2023.csv'
    id_file_lcd2 = file_last_modified.name.split('_')[1]
#     which files contain id for the current lcd_version?
    files_ = [file_ for file_ in version_files[lcd_version] if id_file_lcd2 in file_.name]
    files_lcd_input.extend(files_)
#     print('v2')
else:
    files_lcd_input.extend(file_last_modified)

In [17]:
# string version of filenames from files_lcd_input as vertical list
files_lcd_input_names_str = "\n".join([f_.name for f_ in files_lcd_input])
# files_lcd_input_names_str

### Keep only files that have one or more columns from cols_noaa_processed

#### Which columns are present by file in files_lcd_input?

In [18]:
# read only headers of each file in files_lcd_input
files_columns = {}
for file_ in files_lcd_input:
    try:
#         this is 30x faster than pd.read_csv(file_, index_col=0, nrows=0).columns.tolist()
        with open(file_, 'r') as infile:
            reader = csv.DictReader(infile)
            fieldnames = reader.fieldnames
        files_columns[file_] = sorted(fieldnames)
    except:
        continue

### Create files_usecols containing valid files and columns to be used

#### Is 'DATE' available for every file in files_lcd_input?

In [19]:
# keep only files that have a 'DATE' column - otherwise where is this data supposed to go?
files_usecols = {file_ : cols_ for file_, cols_ in files_columns.items() if 'DATE' in cols_}
files_usecols

{WindowsPath('C:/Users/user/OneDrive/python_envs/noaa-weather-hourly-cli/dev/data/LCD_USW00014939_2023.csv'): ['BackupDirection',
  'BackupDistance',
  'BackupDistanceUnit',
  'BackupElements',
  'BackupElevation',
  'BackupEquipment',
  'BackupLatitude',
  'BackupLongitude',
  'BackupName',
  'CoolingDegreeDaysSeasonToDate',
  'DATE',
  'DailyAverageDewPointTemperature',
  'DailyAverageDryBulbTemperature',
  'DailyAverageRelativeHumidity',
  'DailyAverageSeaLevelPressure',
  'DailyAverageStationPressure',
  'DailyAverageWetBulbTemperature',
  'DailyAverageWindSpeed',
  'DailyCoolingDegreeDays',
  'DailyDepartureFromNormalAverageTemperature',
  'DailyHeatingDegreeDays',
  'DailyMaximumDryBulbTemperature',
  'DailyMinimumDryBulbTemperature',
  'DailyPeakWindDirection',
  'DailyPeakWindSpeed',
  'DailyPrecipitation',
  'DailySnowDepth',
  'DailySnowfall',
  'DailySustainedWindDirection',
  'DailySustainedWindSpeed',
  'DailyWeather',
  'ELEVATION',
  'HeatingDegreeDaysSeasonToDate',
  'H

#### Keep only files that have at least one cols_data column

In [20]:
files_usecols = {file_ : cols_ for file_, cols_ in files_usecols.items()
                 if len(set(cols_).intersection(set(cols_data))) >=1}
files_usecols

{WindowsPath('C:/Users/user/OneDrive/python_envs/noaa-weather-hourly-cli/dev/data/LCD_USW00014939_2023.csv'): ['BackupDirection',
  'BackupDistance',
  'BackupDistanceUnit',
  'BackupElements',
  'BackupElevation',
  'BackupEquipment',
  'BackupLatitude',
  'BackupLongitude',
  'BackupName',
  'CoolingDegreeDaysSeasonToDate',
  'DATE',
  'DailyAverageDewPointTemperature',
  'DailyAverageDryBulbTemperature',
  'DailyAverageRelativeHumidity',
  'DailyAverageSeaLevelPressure',
  'DailyAverageStationPressure',
  'DailyAverageWetBulbTemperature',
  'DailyAverageWindSpeed',
  'DailyCoolingDegreeDays',
  'DailyDepartureFromNormalAverageTemperature',
  'DailyHeatingDegreeDays',
  'DailyMaximumDryBulbTemperature',
  'DailyMinimumDryBulbTemperature',
  'DailyPeakWindDirection',
  'DailyPeakWindSpeed',
  'DailyPrecipitation',
  'DailySnowDepth',
  'DailySnowfall',
  'DailySustainedWindDirection',
  'DailySustainedWindSpeed',
  'DailyWeather',
  'ELEVATION',
  'HeatingDegreeDaysSeasonToDate',
  'H

In [21]:
# reduce files_usecols to only columns used in this process
files_usecols = {file_ : sorted(set(cols_noaa_processed).intersection(set(cols_))) for
                 file_, cols_ in files_usecols.items()}
files_usecols

{WindowsPath('C:/Users/user/OneDrive/python_envs/noaa-weather-hourly-cli/dev/data/LCD_USW00014939_2023.csv'): ['DATE',
  'HourlyAltimeterSetting',
  'HourlyDewPointTemperature',
  'HourlyDryBulbTemperature',
  'HourlyPrecipitation',
  'HourlyPressureChange',
  'HourlyRelativeHumidity',
  'HourlyStationPressure',
  'HourlyVisibility',
  'HourlyWetBulbTemperature',
  'HourlyWindDirection',
  'HourlyWindGustSpeed',
  'HourlyWindSpeed',
  'STATION',
  'Sunrise',
  'Sunset'],
 WindowsPath('C:/Users/user/OneDrive/python_envs/noaa-weather-hourly-cli/dev/data/LCD_USW00014939_2022.csv'): ['DATE',
  'HourlyAltimeterSetting',
  'HourlyDewPointTemperature',
  'HourlyDryBulbTemperature',
  'HourlyPrecipitation',
  'HourlyPressureChange',
  'HourlyRelativeHumidity',
  'HourlyStationPressure',
  'HourlyVisibility',
  'HourlyWetBulbTemperature',
  'HourlyWindDirection',
  'HourlyWindGustSpeed',
  'HourlyWindSpeed',
  'STATION',
  'Sunrise',
  'Sunset'],
 WindowsPath('C:/Users/user/OneDrive/python_envs

### Open and Concatenate all files in files_usecols

In [116]:
df = pd.concat((pd.read_csv(f_, usecols=cols_, parse_dates=['DATE'],
                            index_col='DATE', low_memory=False) for
                f_, cols_ in files_usecols.items()), axis=0)\
                .reset_index().drop_duplicates()
df = df.set_index('DATE', drop=True).sort_index()
df

,STATION,HourlyAltimeterSetting,HourlyDewPointTemperature,HourlyDryBulbTemperature,HourlyPrecipitation,HourlyPressureChange,HourlyRelativeHumidity,HourlyStationPressure,HourlyVisibility,HourlyWetBulbTemperature,HourlyWindDirection,HourlyWindGustSpeed,HourlyWindSpeed,Sunrise,Sunset
DATE,,,,,,,,,,,,,,,
2020-01-01 00:00:00,USW00014939,NaN,-6.1,-2.8,NaN,-1.9,78.0,967.9,NaN,-3.9,220.0,NaN,2.6,NaN,NaN
2020-01-01 00:00:00,USW00014939,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,751.0,1709.0
2020-01-01 00:00:00,USW00014939,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-01-01 00:54:00,USW00014939,1009.8,-7.8,-5.0,0.0,NaN,81.0,967.2,16.093,-5.9,170.0,NaN,2.6,NaN,NaN
2020-01-01 01:54:00,USW00014939,1008.1,-7.8,-4.4,0.0,NaN,77.0,965.5,16.093,-5.5,170.0,NaN,2.6,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-12-31 20:54:00,USW00014939,1027.8,-6.1,-3.3,NaN,1.2,81.0,NaN,16.093,NaN,300,NaN,1.5,NaN,NaN
2023-12-31 21:00:00,USW00014939,NaN,-6.1,-3.3,NaN,1.2,81.0,984.3,16.0,-4.3,300,NaN,1.5,NaN,NaN
2023-12-31 21:54:00,USW00014939,1027.8,-7.8,-5.6,NaN,NaN,85.0,NaN,16.093,NaN,000,NaN,0.0,NaN,NaN


In [117]:
# keep track of the count of raw timestamps prior to processing
n_records_raw = df.shape[0]
# track statistics by column prior to processing, omit 'Sunrise' & 'Sunset' from stats
cols_sunrise_sunset = df.columns.intersection(['Sunrise', 'Sunset']).tolist()
df_stats_pre = df.loc[:, df.columns.difference(cols_sunrise_sunset)].describe()

In [118]:
# df = df[cols_use].copy()
df.shape

(50283, 15)

In [119]:
df.head()

,STATION,HourlyAltimeterSetting,HourlyDewPointTemperature,HourlyDryBulbTemperature,HourlyPrecipitation,HourlyPressureChange,HourlyRelativeHumidity,HourlyStationPressure,HourlyVisibility,HourlyWetBulbTemperature,HourlyWindDirection,HourlyWindGustSpeed,HourlyWindSpeed,Sunrise,Sunset
DATE,,,,,,,,,,,,,,,
2020-01-01 00:00:00,USW00014939,NaN,-6.1,-2.8,NaN,-1.9,78.0,967.9,NaN,-3.9,220.0,NaN,2.6,NaN,NaN
2020-01-01 00:00:00,USW00014939,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,751.0,1709.0
2020-01-01 00:00:00,USW00014939,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-01-01 00:54:00,USW00014939,1009.8,-7.8,-5.0,0.0,NaN,81.0,967.2,16.093,-5.9,170.0,NaN,2.6,NaN,NaN
2020-01-01 01:54:00,USW00014939,1008.1,-7.8,-4.4,0.0,NaN,77.0,965.5,16.093,-5.5,170.0,NaN,2.6,NaN,NaN


### Identify and Display Weather Station Information

In [120]:
# v1 & v2
# identify WBAN station
station_lcd = str(df['STATION'].value_counts().index[0])
station_wban = station_lcd[6:]  #important - this is index for the isd-history table
station_lcd, station_wban

('USW00014939', '14939')

In [121]:
# remove 'STATION', 'REPORT_TYPE', 'SOURCE' columns - not needed anymore
df.drop(columns=['STATION', 'REPORT_TYPE', 'SOURCE'], inplace=True, errors='ignore')

#### Open 'isd-history.csv' containing Station details
This is supplemental, non-essential information.   It is important that any errors with this step do not negatively impact the core process.

In [122]:
isd_history_available = False
file_isd_history = find_latest_file(dir_data, pattern_isd_history_file)
if file_isd_history.is_file():
    isd_history_available = True

In [123]:
df_isd_history = pd.read_csv(file_isd_history, index_col='WBAN',
                             dtype={'WBAN': object}).sort_values(by=['USAF', 'BEGIN'],
                                                             ascending=[True, False])
df_isd_history.sample(5).sort_index()

,USAF,STATION NAME,CTRY,ST,CALL,LAT,LON,ELEV(M),BEGIN,END
WBAN,,,,,,,,,,
00306,949999,YALLOURNSEC,AS,NaN,NaN,-38.180,+146.370,+0155.1,1973-12-31,1982-10-30
10715,999999,COCO SOLO,PM,NaN,MBFS,+09.367,-079.900,+0004.0,1945-05-01,1957-07-18
24119,725835,BATTLE MOUNTAIN AIRPORT,US,NV,KBAM,+40.612,-116.892,+1373.1,1973-01-01,2024-12-15
68201,999999,BYRD STATION,AY,NaN,NaN,-80.017,-119.533,+1542.9,1957-01-10,1972-12-31
94997,726561,WADENA MUNICIPAL AIRPORT,US,MN,KADC,+46.447,-095.212,+0417.6,2006-01-01,2024-12-16


In [124]:
df_isd_history.loc[station_wban].reset_index()

,WBAN,USAF,STATION NAME,CTRY,ST,CALL,LAT,LON,ELEV(M),BEGIN,END
0,14939,725510,LINCOLN MUNICIPAL AIRPORT,US,NE,KLNK,+40.848,-096.765,+0356.7,1942-10-01,2024-12-16
1,14939,999999,LINCOLN AIR BASE,US,NE,KLNK,+40.851,-096.748,+0362.4,1948-01-01,1972-12-31


In [125]:
# df_isd_history.loc[int(station_wban)]
# using station_wban, gather station details from isd history for the record with the most
# recent 'END' date
station_details = dict(df_isd_history.loc[station_wban].reset_index()\
                       .sort_values('END', ascending=False).iloc[0])
# delete df_isd_history - no longer needed
del df_isd_history

In [126]:
# add url to google maps search of lat, lon values to station_details
google_maps_lat_lon_url = """https://maps.google.com/?q={lat},{long}"""
google_maps_url = google_maps_lat_lon_url.format(lat = station_details['LAT'],
                                                long = station_details['LON'])
station_details['GOOGLE MAP'] = google_maps_url
station_details

{'WBAN': '14939',
 'USAF': '725510',
 'STATION NAME': 'LINCOLN MUNICIPAL AIRPORT',
 'CTRY': 'US',
 'ST': 'NE',
 'CALL': 'KLNK',
 'LAT': '+40.848',
 'LON': '-096.765',
 'ELEV(M)': '+0356.7',
 'BEGIN': '1942-10-01',
 'END': '2024-12-16',
 'GOOGLE MAP': 'https://maps.google.com/?q=+40.848,-096.765'}

In [127]:
#     create timestamps for consolidated table df
start_dt = df.index[0]
end_dt = df.index[-1]
start_str = start_dt.strftime('%Y-%m-%d')
end_str = end_dt.strftime('%Y-%m-%d')
start_str, end_str

('2020-01-01', '2023-12-31')

In [128]:
df.dtypes

HourlyAltimeterSetting       float64
HourlyDewPointTemperature    float64
HourlyDryBulbTemperature     float64
HourlyPrecipitation           object
HourlyPressureChange         float64
HourlyRelativeHumidity       float64
HourlyStationPressure        float64
HourlyVisibility              object
HourlyWetBulbTemperature     float64
HourlyWindDirection           object
HourlyWindGustSpeed          float64
HourlyWindSpeed              float64
Sunrise                      float64
Sunset                       float64
dtype: object

In [129]:
cols_dtypes = {'HourlyAltimeterSetting': 'float64',
   'HourlyDewPointTemperature': 'float64',
    'HourlyDryBulbTemperature': 'float64',
    'HourlyPrecipitation': 'float64',
    'HourlyPressureChange': 'float64',
    'HourlyRelativeHumidity': 'Int64',
    'HourlyStationPressure': 'float64',
    'HourlyVisibility': 'float64',
    'HourlyWetBulbTemperature': 'float64',
    'HourlyWindDirection': 'Int64',
    'HourlyWindGustSpeed': 'float64',
    'HourlyWindSpeed': 'float64',
    'Sunrise': 'float64',
    'Sunset': 'float64'}
#     'Sunrise': 'datetime64[s]',
#     'Sunset': 'datetime64[s]'}
# keep only items with corresponding column in df
cols_dtypes = {k : v for k, v in cols_dtypes.items() if k in df.columns}

In [130]:
# enforce dtypes for df
# does this need to be done on pre- df?  Is post- df processing enough?
for col_, type_ in cols_dtypes.items():
    if type_ == 'float64':
        df[col_] = pd.to_numeric(df[col_], errors='coerce')
        try:
            df[col_] = df[col_].astype(type_)
        except:
            pass
    elif type_ == 'Int64':
        df[col_] = pd.to_numeric(df[col_], errors='coerce')
        try:
            df[col_] = df[col_].astype(type_)
        except:
            pass

In [131]:
df.dtypes

HourlyAltimeterSetting       float64
HourlyDewPointTemperature    float64
HourlyDryBulbTemperature     float64
HourlyPrecipitation          float64
HourlyPressureChange         float64
HourlyRelativeHumidity         Int64
HourlyStationPressure        float64
HourlyVisibility             float64
HourlyWetBulbTemperature     float64
HourlyWindDirection            Int64
HourlyWindGustSpeed          float64
HourlyWindSpeed              float64
Sunrise                      float64
Sunset                       float64
dtype: object

#### Resolve duplicate datetime index values

In [132]:
df = df.groupby(level=0).mean()
df.shape

(48807, 14)

#### Extract Sunrise and Sunset by date in to dictionaries

to be applied
to df_out towards end of script.
Drop Sunrise and Sunset columns after extraction here
date_sunrise, date_sunset
The source data provides only one record per day and rest are NaN
additionally, resampling datetime values was causing dtype challenges

In [138]:
def datetime_from_HHMM(x):
    """Returns """
    
    time = pd.to_datetime(pd.to_numeric(x.dropna().ffill()\
                             .bfill()), format = '%H%M')
    YMDHM = pd.DataFrame({'Year': x.index.year,
                         'Month' : x.index.month,
                        'Day' : x.index.day,
                        'Hour' : time.dt.hour,
                         'Minute' : time.dt.minute})

    return pd.to_datetime(YMDHM)
    

In [139]:
df[cols_sunrise_sunset].dropna()

,Sunrise,Sunset
DATE,,
2020-01-01,751.0,1709.0
2020-01-02,752.0,1710.0
2020-01-03,752.0,1711.0
2020-01-04,752.0,1712.0
2020-01-05,752.0,1713.0
...,...,...
2023-12-27,750.0,1705.0
2023-12-28,750.0,1706.0
2023-12-29,751.0,1707.0


In [140]:
date_sunrise = datetime_from_HHMM(df['Sunrise'].dropna())
date_sunset = datetime_from_HHMM(df['Sunset'].dropna())

#### are there timestamps that have a high count of null values?
In v1 LCD files the '23:59:00' timestamp is suspect and appears to only be a placeholder
for posting sunrise/sunset times.  Important that this step be done after
forward filling sunrise/sunset values. 
V2 LCD files do not seem to have the '23:59:00' timestamp issue.

In [143]:
# n_records_hourly_approx = int(df.shape[0]/(24))
# n_max_null = pct_null_timestamp_max * n_records_hourly_approx
n_max_null = int(pct_null_timestamp_max * df.shape[0])
n_max_null

24403

In [144]:
temp = df.loc[:, df.columns.difference(cols_sunrise_sunset)]
df_nan_ts = temp.groupby(temp.index.time).apply(lambda x: x.isna().sum()\
                            .gt(n_max_null)).all(axis=1)
times_nan = df_nan_ts.loc[df_nan_ts].index.tolist()
del temp
del df_nan_ts

In [145]:
# remove records for timestamps with a high percentage of Null values.
# note that the '23:59:00' timestamp is suspect and appears to only be a placeholder
# for posting sunrise/sunset times.  Important that this step be done after
# forward filling sunrise/sunset values.
filter_nan_times = pd.Series(df.index.time).isin(times_nan).values
df = df.loc[~filter_nan_times]
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 48807 entries, 2020-01-01 00:00:00 to 2023-12-31 23:54:00
Data columns (total 14 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   HourlyAltimeterSetting     42846 non-null  float64
 1   HourlyDewPointTemperature  48754 non-null  float64
 2   HourlyDryBulbTemperature   48772 non-null  float64
 3   HourlyPrecipitation        34925 non-null  float64
 4   HourlyPressureChange       17481 non-null  float64
 5   HourlyRelativeHumidity     48754 non-null  Float64
 6   HourlyStationPressure      48642 non-null  float64
 7   HourlyVisibility           43696 non-null  float64
 8   HourlyWetBulbTemperature   48623 non-null  float64
 9   HourlyWindDirection        43923 non-null  Float64
 10  HourlyWindGustSpeed        8119 non-null   float64
 11  HourlyWindSpeed            48579 non-null  float64
 12  Sunrise                    1461 non-null   float64
 13  Sunset     

In [146]:
test = df.between_time('23:01:00', '00:59:00').select_dtypes(include=[int, float])
test.groupby(test.index.time).sum()

,HourlyAltimeterSetting,HourlyDewPointTemperature,HourlyDryBulbTemperature,HourlyPrecipitation,HourlyPressureChange,HourlyRelativeHumidity,HourlyStationPressure,HourlyVisibility,HourlyWetBulbTemperature,HourlyWindDirection,HourlyWindGustSpeed,HourlyWindSpeed,Sunrise,Sunset
00:00:00,2019.6,5909.6,13355.7,1.7,429.2,102080.0,1390264.6,3130.854,9881.6,249289.0,0.0,5134.7,896240.0,2683614.0
00:01:00,9151.4,73.1,97.4,0.0,0.0,760.0,8765.3,100.179,84.5,1820.0,18.0,42.8,0.0,0.0
00:02:00,8103.7,29.9,42.3,0.7,0.0,720.0,7761.7,61.556,36.8,1380.0,26.8,41.7,0.0,0.0
00:03:00,8129.1,52.2,76.7,0.5,0.0,648.0,7786.0,113.456,66.6,2100.0,14.4,43.8,0.0,0.0
00:04:00,6056.2,53.9,65.0,1.4,0.0,531.0,5800.4,73.223,59.2,500.0,11.3,32.5,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23:54:00,1482354.9,6054.4,13670.9,100.1,438.3,104213.0,1411051.9,22073.600,10033.1,252780.0,1994.1,5220.0,0.0,0.0
23:56:00,2024.7,32.8,36.6,0.2,0.0,178.0,1939.2,18.507,34.2,390.0,13.9,10.8,0.0,0.0
23:57:00,1010.8,-10.0,-8.9,0.0,0.0,92.0,968.2,0.000,-9.2,360.0,0.0,1.5,0.0,0.0
23:58:00,4041.0,41.1,45.0,2.9,0.0,374.0,3870.2,17.702,43.0,460.0,12.9,20.1,0.0,0.0


In [147]:
if is_development:
    df.groupby(df.index.time).count().max(axis=1).sort_index().plot(figsize=(11,3),
   title='Count of Maximum Non-Null Readings per Timestamp')

In [148]:
# df.loc[filter_nan_times]

In [149]:
#     Check what percentage of data has null data and print to screen
df_pct_null_data = pd.DataFrame({'Percent N/A': df.isnull().sum().divide(len(df)).round(3)})
df_pct_null_data_pre_formatted = df_pct_null_data['Percent N/A']\
                            .apply(lambda n: '{:,.1%}'.format(n))
# remove 'Hourly' prefix for display only
col_rename_remove_hourly = {col_ : col_.replace('Hourly', '') for
                            col_ in df_pct_null_data_pre_formatted.index}

#### Display Station Details

In [150]:
# exclude station lifetime history dates - could cause confusion
station_details_exclude = ['BEGIN', 'END']
station_details_display = {k_ : v_ for k_, v_ in station_details.items() if
                           k_ not in station_details_exclude}

In [151]:
print('--------------------------------------------')
print('------ ISD Weather Station Properties ------')
print('--------------------------------------------')
for k_, v_ in station_details_display.items():
    print("{:<15} {:<10}".format(k_, v_))
print('\n')

--------------------------------------------
------ ISD Weather Station Properties ------
--------------------------------------------
WBAN            14939     
USAF            725510    
STATION NAME    LINCOLN MUNICIPAL AIRPORT
CTRY            US        
ST              NE        
CALL            KLNK      
LAT             +40.848   
LON             -096.765  
ELEV(M)         +0356.7   
GOOGLE MAP      https://maps.google.com/?q=+40.848,-096.765




In [152]:
# message_pct_null_data = f"""Percent Missing Values by Column for LCD source file '{file_lcd_input.name}' for USAF station {station_usaf} at '{station_details['STATION NAME']}' from {start_str} to {end_str}."""
# message_pct_null_data = """Percent Missing Values by Column for LCD source file(s):
# {files_lcd_input_names_str}\n\nFor USAF station {station_usaf} at '{station_name}'
# from {start_str} to {end_str}."""
message_pct_null_data = """Percent Missing Values by Column from {start_str} to {end_str} for LCD source file(s):\n
{files_lcd_input_names_str}"""
message_ = message_pct_null_data.format(files_lcd_input_names_str = files_lcd_input_names_str,
                            station_usaf = station_details['USAF'],
                            station_name = station_details['STATION NAME'],
                            start_str = start_str,
                            end_str = end_str)

print(message_)

Percent Missing Values by Column from 2020-01-01 to 2023-12-31 for LCD source file(s):

LCD_USW00014939_2023.csv
LCD_USW00014939_2022.csv
LCD_USW00014939_2021.csv
LCD_USW00014939_2020.csv


In [153]:
print('--------------------------------------------------------------')
print('------ Percent Null Values by Column Before Processing -------')
print('--------------------------------------------------------------')
display(df_pct_null_data_pre_formatted.rename(index=col_rename_remove_hourly))

--------------------------------------------------------------
------ Percent Null Values by Column Before Processing -------
--------------------------------------------------------------


AltimeterSetting       12.2%
DewPointTemperature     0.1%
DryBulbTemperature      0.1%
Precipitation          28.4%
PressureChange         64.2%
RelativeHumidity        0.1%
StationPressure         0.3%
Visibility             10.5%
WetBulbTemperature      0.4%
WindDirection          10.0%
WindGustSpeed          83.4%
WindSpeed               0.5%
Sunrise                97.0%
Sunset                 97.0%
Name: Percent N/A, dtype: object

### Define Policy dictionaries for filling of Nulls and Resampling

#### How to handle Null values - col_null_policy

In [154]:
# create dictionary 'col_null_policy' to manually guide the resolution of null values by column
# in general, data that is expected to have a non-zero/non-null reading at all times
# will have nulls filled through interpolation.  Data that normally has 0 or null readings
# like 'HourlyPrecipitation' will have null values filled with 0 in the interest of having
# a 100% clean dataset to facilitate manual analysis.
col_null_policy = {'HourlyAltimeterSetting': pd.Series.interpolate,
 'HourlyDewPointTemperature': pd.Series.interpolate,
 'HourlyDryBulbTemperature': pd.Series.interpolate,
 'HourlyPrecipitation': lambda x: x.fillna(0),
 'HourlyPressureChange': pd.Series.interpolate,
 'HourlyRelativeHumidity': pd.Series.interpolate,
 'HourlyStationPressure': pd.Series.interpolate,
 'HourlyVisibility': pd.Series.interpolate,
 'HourlyWetBulbTemperature': pd.Series.interpolate,
 'HourlyWindDirection': pd.Series.interpolate,
 'HourlyWindGustSpeed': lambda x: x,
 'HourlyWindSpeed': pd.Series.interpolate,
 'Sunrise': pd.Series.ffill,
 'Sunset': pd.Series.bfill}
# keep only items with corresponding column in df
col_null_policy = {k : v for k, v in col_null_policy.items() if k in df.columns}
# assert len(df.columns.intersection(col_null_policy.keys())) == df.shape[1]

In [155]:
df.dtypes

HourlyAltimeterSetting       float64
HourlyDewPointTemperature    float64
HourlyDryBulbTemperature     float64
HourlyPrecipitation          float64
HourlyPressureChange         float64
HourlyRelativeHumidity       Float64
HourlyStationPressure        float64
HourlyVisibility             float64
HourlyWetBulbTemperature     float64
HourlyWindDirection          Float64
HourlyWindGustSpeed          float64
HourlyWindSpeed              float64
Sunrise                      float64
Sunset                       float64
dtype: object

In [156]:
# col = 'HourlyAltimeterSetting'
# # df[col].resample('H').agg(col_null_policy[col])

# pd.to_numeric(df[col], errors='coerce').dropna().resample(freqstr).mean()

# col = 'HourlyAltimeterSetting'
# # df[col].resample('H').agg(col_null_policy[col])

# pd.to_numeric(df[col], errors='coerce').dropna().resample(freqstr).mean()

In [157]:
# create dictionary 'col_resample_policy' to manually guide resampling method by column

col_resample_policy = {'HourlyAltimeterSetting': 'mean',
 'HourlyDewPointTemperature': 'mean',
 'HourlyDryBulbTemperature': 'mean',
 'HourlyPrecipitation': 'mean',
 'HourlyPressureChange': 'mean',
 'HourlyRelativeHumidity': 'mean',
 'HourlyStationPressure': 'mean',
 'HourlyVisibility': 'mean',
 'HourlyWetBulbTemperature': 'mean',
 'HourlyWindDirection': 'mean',
 'HourlyWindGustSpeed': 'mean',
 'HourlyWindSpeed': 'mean',
#  'Sunrise': pd.Series.mode,
#  'Sunset': pd.Series.mode}
 'Sunrise': 'mean',
 'Sunset': 'mean'}
# keep only items with corresponding column in df
col_resample_policy = {k : v for k, v in col_resample_policy.items() if k in df.columns}
# assert len(df.columns.intersection(col_resample_policy.keys())) == df.shape[1]

In [158]:
# %%timeit
# col_ = 'Sunrise'
# df[col_].dropna().resample(freqstr).agg(col_resample_policy[col_])

# %%timeit
# col_ = 'Sunrise'
# df[col_].dropna().resample(freqstr).agg(col_resample_policy[col_])

# %%timeit
# col_ = 'Sunrise'
# df[col_].dropna().resample(freqstr).agg(lambda x: x.mode())

# %%timeit
# col_ = 'Sunrise'
# df[col_].dropna().resample(freqstr).agg(lambda x: x.median())

# %%timeit
# col_ = 'Sunrise'
# df[col_].dropna().resample(freqstr).agg(pd.Series.median)

# %%timeit
# col_ = 'Sunrise'
# df[col_].dropna().resample(freqstr).agg(pd.Series.median)

In [159]:
%%time
dfs = {}
# for col_ in df.columns:
for col_ in df.columns.difference(cols_sunrise_sunset):
    print(col_)
#     if df[col_].dtype.name == 'datetime64[ns]':
#         df[col_].resample(freqstr).agg(col_resample_policy[col_])
#     else:
#     df[col_].agg(col_null_policy[col_])
#         pd.to_numeric(df[col], errors='coerce').dropna().resample(freqstr).agg(col_resample_policy[col_])
    dfs[col_] = df[col_].dropna().resample(freqstr).agg(col_resample_policy[col_])

HourlyAltimeterSetting
HourlyDewPointTemperature
HourlyDryBulbTemperature
HourlyPrecipitation
HourlyPressureChange
HourlyRelativeHumidity
HourlyStationPressure
HourlyVisibility
HourlyWetBulbTemperature
HourlyWindDirection
HourlyWindGustSpeed
HourlyWindSpeed
CPU times: total: 93.8 ms
Wall time: 80.5 ms


In [160]:
df_out = pd.concat(dfs, axis=1).drop_duplicates()
del dfs
del df
df_out

,HourlyAltimeterSetting,HourlyDewPointTemperature,HourlyDryBulbTemperature,HourlyPrecipitation,HourlyPressureChange,HourlyRelativeHumidity,HourlyStationPressure,HourlyVisibility,HourlyWetBulbTemperature,HourlyWindDirection,HourlyWindGustSpeed,HourlyWindSpeed
DATE,,,,,,,,,,,,
2020-01-01 00:00:00,1009.8,-6.95,-3.90,0.0,-1.9,79.5,967.55,16.0930,-4.9,195.0,NaN,2.60
2020-01-01 01:00:00,1008.1,-7.80,-4.40,0.0,NaN,77.0,965.50,16.0930,-5.5,170.0,NaN,2.60
2020-01-01 02:00:00,1007.5,-7.80,-4.40,0.0,-3.4,77.0,964.90,16.0930,-5.5,170.0,NaN,2.60
2020-01-01 03:00:00,1006.8,-7.80,-3.30,0.0,NaN,71.0,964.20,16.0930,-4.8,150.0,NaN,2.10
2020-01-01 04:00:00,1005.4,-6.70,-2.20,0.0,NaN,72.0,962.90,16.0930,-3.7,210.0,NaN,3.60
...,...,...,...,...,...,...,...,...,...,...,...,...
2023-12-31 19:00:00,1027.4,-5.85,-2.20,NaN,NaN,76.5,NaN,16.0930,NaN,330.0,NaN,3.60
2023-12-31 20:00:00,1027.8,-6.10,-3.05,NaN,1.2,79.5,NaN,16.0930,NaN,310.0,NaN,1.50
2023-12-31 21:00:00,1027.8,-6.95,-4.45,NaN,1.2,83.0,984.30,16.0465,-4.3,150.0,NaN,0.75


# Resolve Sunrise/Sunset

In [162]:
# apply date_sunrise/sunset to df_out index to create sunrise/sunset columns
df_out['Sunrise'] = pd.Series(df_out.index.map(date_sunrise), index=df_out.index).ffill()
df_out['Sunset'] = pd.Series(df_out.index.map(date_sunset), index=df_out.index).ffill()

In [172]:
# enforce dtypes for df_out
# does this need to be done on pre- df_out?  Is post- df_out processing enough?
for col_, type_ in cols_dtypes.items():
    if type_ == 'float64':
        df_out[col_] = pd.to_numeric(df_out[col_], errors='coerce')
        try:
            df_out[col_] = df_out[col_].astype(type_)
        except:
            pass
    elif type_ == 'Int64':
        df_out[col_] = pd.to_numeric(df_out[col_], errors='coerce')
        try:
            df_out[col_] = df_out[col_].astype(type_)
        except:
            pass
    elif type_ == 'datetime64[s]':
        df_out[col_] = pd.to_datetime(df_out[col_], errors='coerce')
        try:
            df_out[col_] = df_out[col_].astype(type_)
        except:
            pass

In [173]:
df_stats_pre 

,HourlyAltimeterSetting,HourlyDewPointTemperature,HourlyDryBulbTemperature,HourlyPressureChange,HourlyRelativeHumidity,HourlyStationPressure,HourlyWetBulbTemperature,HourlyWindGustSpeed,HourlyWindSpeed
count,42846.000000,48754.000000,48772.000000,17481.000000,48754.000000,48642.000000,48623.000000,8119.000000,48579.000000
mean,1016.126364,4.494400,11.444915,0.000440,66.181872,973.275632,7.978259,13.330915,4.498030
std,7.600758,11.259973,12.294248,1.500063,19.349224,7.333802,10.608199,3.278796,2.998525
min,982.400000,-37.200000,-33.900000,-8.700000,10.000000,940.700000,-34.000000,7.200000,0.000000
25%,1011.500000,-4.400000,1.100000,-0.900000,52.000000,968.800000,-0.200000,11.300000,2.100000
50%,1015.900000,3.900000,11.700000,0.000000,69.000000,973.100000,8.100000,12.900000,4.100000
75%,1020.700000,15.000000,21.700000,0.900000,82.000000,977.600000,17.600000,14.900000,6.200000
max,1044.000000,27.200000,40.600000,11.800000,100.000000,1000.200000,29.300000,41.700000,26.300000


In [174]:
df_stats_post = df_out[df_stats_pre.columns].describe()
df_stats_post

,HourlyAltimeterSetting,HourlyDewPointTemperature,HourlyDryBulbTemperature,HourlyPressureChange,HourlyRelativeHumidity,HourlyStationPressure,HourlyWetBulbTemperature,HourlyWindGustSpeed,HourlyWindSpeed
count,34977.000000,34974.000000,34977.000000,17481.000000,34974.0,34878.000000,34874.000000,6651.000000,34861.000000
mean,1016.471511,4.339591,12.050943,0.000440,63.018099,973.569894,8.195601,12.974051,4.376544
std,7.490799,11.302829,12.407358,1.500063,19.029542,7.235153,10.603644,2.994208,2.905698
min,984.350000,-37.200000,-33.900000,-8.700000,10.0,942.575000,-34.000000,7.200000,0.000000
25%,1011.900000,-4.400000,2.200000,-0.900000,49.0,969.100000,0.000000,10.800000,2.100000
50%,1016.300000,3.900000,12.650000,0.000000,65.0,973.400000,8.300000,12.400000,4.100000
75%,1021.000000,15.000000,22.200000,0.900000,79.0,978.000000,17.800000,14.400000,6.200000
max,1044.000000,27.200000,40.600000,11.800000,100.0,1000.200000,29.300000,31.933333,17.833333


In [175]:
# TO-DO:  Overall pre- post- statistics check
df_stats_pre.sub(df_stats_post).div(df_stats_pre).mean().mean()
# df_stats_pre.eq(df_stats_post).sum().sum()
# df_stats_pre.size

0.04220432878694551

# STOPPED HERE

#### Resolve duplicate datetime index values

In [ ]:
df.columns

In [ ]:
df.select_dtypes(exclude=[object]).columns

In [ ]:
df.groupby(df.index).mean()

In [ ]:
tt = df.iloc[:50].select_dtypes(exclude=[object])
tt.groupby(tt.index).mean()

In [ ]:
df_out[cols_sunrise_sunset]

In [ ]:
df_out[cols_sunrise_sunset].info()

In [ ]:
if is_development:
    df_out[cols_sunrise_sunset].diff(axis=1).iloc[:,-1].dt.total_seconds().div(3600).plot()

In [ ]:
# df.agg(col_null_policy).astype(cols_dtypes)
# df.agg(col_null_policy).isna().sum()

In [ ]:
# transform remaining null values per the 'col_null_policy'
# df = df.agg(col_null_policy).astype(cols_dtypes)
# display(pd.DataFrame({'Percent N/A': df.isnull().sum().divide(len(df))\
#                       .round(3)}).apply(lambda n: '{:,.1%}'.format(n)))

In [ ]:
#     Check what percentage of data has null data and print to screen
df_pct_null_data_post = pd.DataFrame({'Percent N/A': df_out.isnull().sum().divide(len(df_out)).round(3)})
df_pct_null_data_post_formatted = df_pct_null_data_post['Percent N/A'].apply(lambda n: '{:,.1%}'.format(n))
display(df_pct_null_data_post_formatted.rename(index = col_rename_remove_hourly))

In [ ]:
display(df_pct_null_data_pre_formatted.rename(index=col_rename_remove_hourly))

In [ ]:
df_pct_null_data_pre_formatted

In [ ]:
for col_ in df.columns:
    print(col_)
    df[col_].resample(freqstr)

In [ ]:
df.dtypes

In [ ]:
pd.to_numeric(df['HourlyVisibility'])

In [ ]:
pd.to_datetime(df['Sunrise'])

In [ ]:
df.resample(freqstr).mean()

In [ ]:
#     Resample data to produce evenly spaced interval data
# df = df.resample(freqstr).median()
df = df.resample(freqstr).mean()

### Quality Control

#### No duplicate records

#### Complete and consistent timestamps

#### correct dtypes

#### Nan values only in columns permitted to have NaN

## Create Additional Columns (not implemented)

In [ ]:
# #     assign seasons to df in new 'season' column
#     df = weather_assign_season(df)
# #     assign weekend or weekday tag to each day
#     df['weekday'] = 'weekday'
#     df.loc[~df.index.weekday.isin(list(range(0,5))),'weekday'] = 'weekend'
#     df['weekday'] = df['weekday'].astype('category')   # keep memory size to a minimum
# #     Add True/False Peak Coincident column
#     df = is_month_and_hour_in_list(df, new_column='Peak Coincident', months=[6,7,8,9],hours=[14,15,16,17])
# #     Add holiday True/False:
#     years = list(df.index.year.unique())
#     try:
#         holiday_dates = generate_list_of_holidays(years=years)
#         holiday_YMD = []
#         for d in holiday_dates:
#             holiday_YMD.append(datetime.datetime(d.year, d.month, d.day).date())
#         df['holiday'] = [dat in holiday_YMD for dat in df.index.date]
# #         alternatively:
# # holiday_dates = generate_list_of_holidays(years=df.index.year.unique().to_list())
# # df['Date_copy_delete'] = df.index.date.copy()
# # df['holiday'] = df.Date_copy_delete.apply(lambda date: date in holiday_dates)
# # df.drop(columns=['Date_copy_delete'], inplace=True)
#     except:
#         print("Holidays were not tagged.  Veritfy it 'Holidays' python package is installed")
# #     Add temperature bins
#     temp_bins = list(range(-30,110,5))
#     df['DryBulbTemperature_bin'] = pd.cut(df['HourlyDryBulbTemperature'],bins=temp_bins)

### Export df to Disk

#### Rename Columns - remove 'hourly' from names

#### Name export file 

In [ ]:
# city = input("Input city or project name:  ")
# export_name = "{}_".format(city) + start_str + "_to_" + end_str + f"_{df.index.freqstr}.csv"
# export_name

In [ ]:
file_output_format.format(STATION_NAME = 'billy',
                         )

In [ ]:
file_out_name = f"{station_details['']city) + start_str + "_to_" + end_str + f"_{df.index.freqstr}.csv"
file_out = dir_cwd / file_out_name
file_out_name

In [ ]:
# os.path.join(os.path.dirname(file),export_name)

In [ ]:
[col.replace('Hourly','') for col in df.columns]

In [ ]:
df.columns

In [ ]:
df.to_csv(os.path.join(os.path.dirname(file),export_name))
# os.path.join(os.path.dirname(directory),f'All_M_V_data {todays_date}.csv')

# END